In [1]:
# pip install jieba

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict, namedtuple, OrderedDict
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution
import os
from io import BytesIO
from itertools import chain
import random


# BOW

## import data

In [2]:
# read in data 
import json
l=0

# training set
cn_train=[]
en_train=[]
with open('data/translation2019zh_train.json', encoding='utf-8') as f:
    while l<100000:
        line=f.readline()
        j=json.loads(line)
        cn_train.append(j["chinese"])
        en_train.append(j["english"])
        l+=1
    f.close()
    
# test set
l=0
cn_test=[]
en_test=[]
with open('data/translation2019zh_valid.json', encoding='utf-8') as f:
    while l<10:
        line=f.readline()
        j=json.loads(line)
        cn_test.append(j["chinese"])
        en_test.append(j["english"])
        l+=1
    f.close()

In [3]:
import jieba

def seperate(cn_list):
    cn_words=[]
    for s in cn_list:
        words=[]
        w=jieba.cut(s)
        for word in w:
            words.append(word)
        cn_words.append(words)
    return cn_words
cn_sep_test=seperate(cn_test)  
cn_words = seperate(cn_train)
cn_words[0],cn_sep_test[0]


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\52909\AppData\Local\Temp\jieba.cache
Loading model cost 0.640 seconds.
Prefix dict has been built successfully.


(['为了',
  '更好',
  '的',
  '锐度',
  '，',
  '但是',
  '附带',
  '的',
  '会',
  '多一些',
  '颗粒',
  '度',
  '，',
  '可以',
  '使用',
  '这个',
  '显影剂',
  '的',
  '1',
  '：',
  '1',
  '稀释液',
  '。'],
 ['美国', '缓慢', '地', '开始', '倾听', '，', '但', '并非', '没有', '艰难曲折', '。'])

In [4]:
ch1=[]
for s in cn_words:
    for w in s:
        ch1.append(w)

In [5]:
# read in dict
import pandas as pd
import re

def endict():
    dic={}
    dictionary = pd.read_csv('dict/all.txt',header = None)
    for row in dictionary.itertuples():
        result=[]
        translations = re.findall(r"(?:\w|\s)*",row[2])
        for trans in translations:
            if trans!="" and trans!=" ":
                result.append(trans)
        dic[row[1]]=result
    return dic

endict=endict()

endict    

{'为了': ['for', 'for the sake of'],
 '更好': ['better', 'so much the better'],
 '的': ['of', 'pure', 'target'],
 '锐度': ['acutance'],
 '但是': ['but', 'yet'],
 '附带': ['cum', 'parenthesis', 'attach', 'obiter'],
 '会': ['be able to', 'meet', 'understand'],
 '颗粒': ['particle'],
 '度': ['limit', 'spend'],
 '可以': ['may'],
 '使用': ['use', 'employ'],
 '这个': ['this'],
 '显影剂': ['eikonogen', 'photographic developer'],
 '稀释液': ['diluent'],
 '他': ['he'],
 '还': ['repay'],
 '把': ['control', 'a handful of'],
 '宣扬': ['publicity', 'advocate'],
 '自己': ['oneself'],
 '思想': ['thought'],
 '所谓': ['same', 'what is called'],
 '绿皮书': ['Green Book'],
 '称作': ['be called'],
 '新': ['new'],
 '福音书': ['evangel', 'the gospel'],
 '微风': ['breeze', 'gentle breeze'],
 '推着': ['push'],
 '我': ['I'],
 '去': ['go', 'leave', 'apart from'],
 '爱抚': ['show tender care for'],
 '它': ['it'],
 '长耳朵': ['long ears'],
 '它们': ['they'],
 '先烈': ['martyr'],
 '们': ['plural marker for pronouns and a few animate nouns'],
 '鲜血': ['blood'],
 '是': ['are', 'ri

In [6]:
from collections import Counter, defaultdict, namedtuple, OrderedDict
# en_words is a related list to cn_words, has same structure, same number of word per sentence
en_words=[]
punctuation = "，！？。＂＃＄％＆＇（）＊＋－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏"
re_punctuation = "[{}]+".format(punctuation)
# count frequency of ZH-EN translation
# for each chinese words, first look up dictionary, find out all possible translated english words
# then look up all possible translation in parallel text, if found, count+1
def counter(cn_words, en):
    d = defaultdict(lambda: defaultdict(int))
    # chinese sentences, english sentences
    for cs, es in zip(cn_words, en):
        en_w=[]
        # for every word in chinese sentences
        for w in cn_words[0]:
    
            # look up dict, get its english word(list)
            if(re.match(re_punctuation, w, flags=0)):
                if(w=='，'):
                    en_w.append(',')
                if(w=='。'):
                    en_w.append('.')
                if(w=='？'):
                    en_w.append('?')
                if(w=='：'):
                    en_w.append(':')
                if(w=='”'):
                    en_w.append('\"')
                if(w=='“'):
                    en_w.append('\"')
                if(w=='；'):
                    en_w.append(';')
                if(w=='‘'):
                    en_w.append('\'')
                if(w=='’'):
                    en_w.append('\'')
                continue
            if(re.match("[a-zA-Z0-9]",w,flags=0)):
                en_w.append(w)
                continue
#             print(w)
            translate=endict.get(w)
            # if translation exits
            if translate!=None:
                # for every translation
                t=0 
                found=False
                while t<len(translate):
                    if es.find(translate[t])!=-1:
                        d[w][translate[t]]+=1
                        en_w.append(translate[t])
                        found=True
        #                 print(found,translate[t])
                        break
                    t+=1
                if found==False:
                    en_w.append(translate[0])
            else:
                en_w.append('<MISSINGWORD>')
        en_words.append(en_w)                
    return d

word_counts = counter(cn_words, en_train)

                        

In [7]:
en_words[0]
#len(cn_words[0])

['for',
 'better',
 'of',
 'acutance',
 ',',
 'but',
 'cum',
 'of',
 'be able to',
 '<MISSINGWORD>',
 'particle',
 'limit',
 ',',
 'may',
 'use',
 'this',
 'eikonogen',
 'of',
 '1',
 ':',
 '1',
 'diluent',
 '.']

In [8]:
origin={}
for word in endict:
    if type(endict.get(word))== list:
        try:
            endict.get(word)[0]
            origin[word]=endict.get(word)[0]
        except:
            ;
#         origin[word]=endict.get(word)[0]
    else:
        origin[word]=endict.get(word)
origin

mfc_table=origin
mfc_table.update((word, max(tags.keys(), key=lambda key: tags[key])) for word, tags in word_counts.items())
mfc_table

{'为了': 'for',
 '更好': 'better',
 '的': 'of',
 '锐度': 'acutance',
 '但是': 'but',
 '附带': 'cum',
 '会': 'meet',
 '颗粒': 'particle',
 '度': 'limit',
 '可以': 'may',
 '使用': 'use',
 '这个': 'this',
 '显影剂': 'eikonogen',
 '稀释液': 'diluent',
 '他': 'he',
 '还': 'repay',
 '把': 'control',
 '宣扬': 'publicity',
 '自己': 'oneself',
 '思想': 'thought',
 '所谓': 'same',
 '绿皮书': 'Green Book',
 '称作': 'be called',
 '新': 'new',
 '福音书': 'evangel',
 '微风': 'breeze',
 '推着': 'push',
 '我': 'I',
 '去': 'go',
 '爱抚': 'show tender care for',
 '它': 'it',
 '长耳朵': 'long ears',
 '它们': 'they',
 '先烈': 'martyr',
 '们': 'plural marker for pronouns and a few animate nouns',
 '鲜血': 'blood',
 '是': 'are',
 '了': 'end',
 '最后': 'finally',
 '在': 'in',
 '月': 'month',
 '日': 'sun',
 '湖人': 'Lakers',
 '将': 'general',
 '前往': 'leave for',
 '汽车城': 'Motown',
 '底特律': 'Detroit',
 '挑战': 'challenge',
 '活塞队': 'Detroit Pistons',
 '活塞': 'piston',
 '近来': 'recently',
 '东部': 'east',
 '排名': 'ranking',
 '第二': 'second',
 '真是': 'certainly',
 '天造地设': 'heavenly',
 '一对': 'twain'

In [9]:
FakeState = namedtuple('FakeState', 'name')

class MFCTagger:
    missing = FakeState(name = '<MISSING>')
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        en_w=list()
        
        for w in seq:
            if(re.match(re_punctuation, w, flags=0)):       
                continue
            if(re.match("[a-zA-Z0-9\s]",w,flags=0)): 
                if(w!="nan"):
                    continue
#             print(self.table[w])
            en_w.append(self.table[w])
        en_w=list(enumerate(["<start>"]+en_w+["<end>"]))
#         ret=list(enumerate(["<start>"] + [self.table[w] for w in seq if (not (re.match(re_punctuation,w,flags=0) and re.match("[a-zA-Z0-9]",w,flags=0)))] + ["<end>"]))
        
        return 0.,en_w

In [10]:
mfc_model = MFCTagger(mfc_table)

In [11]:
def replace_unknown(sequence):
    return [w if w in ch1 else 'nan' for w in sequence]
def simplify_decoding(X, model):    
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]


In [12]:
for sentence,es in zip(seperate(cn_test)[0:5],en_test[0:5]):
    #print("Sentence Key: {}\n".format(sentence))
    print(tuple(sentence))
#     sentenceinput=re.sub(re_punctuation, "", sentence)
#     sentenceinput=re.sub("[a-zA-Z0-9]","",sentenceinput)
    print("Sentence: {}\n".format(sentence))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(tuple(sentence), mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(es)
    print("\n")

('美国', '缓慢', '地', '开始', '倾听', '，', '但', '并非', '没有', '艰难曲折', '。')
Sentence: ['美国', '缓慢', '地', '开始', '倾听', '，', '但', '并非', '没有', '艰难曲折', '。']

Predicted labels:
-----------------
['America', 'tardiness', 'land', 'initiation', 'give ear to', 'but', 'really not', 'without', '<MISSING>']

Actual labels:
--------------
Slowly and not without struggle, America began to listen.


('直到', '四年', '前', '我', '才', '有', '了', '一本', '词典', '。', '我', '使用', '的', '是', '用', '89', '美分', '在', 'K', '市场', '里', '买来', '的', '一本', '韦氏', '小', '词典', '。', '我', '从来不', '使用', '单词', '处理程序', '。')
Sentence: ['直到', '四年', '前', '我', '才', '有', '了', '一本', '词典', '。', '我', '使用', '的', '是', '用', '89', '美分', '在', 'K', '市场', '里', '买来', '的', '一本', '韦氏', '小', '词典', '。', '我', '从来不', '使用', '单词', '处理程序', '。']

Predicted labels:
-----------------
['till', 'quadrennium', 'before', 'I', 'ability', 'adsum', 'end', 'a volume', 'dictionaries', 'I', 'use', 'of', 'are', 'use', '<MISSING>', 'in', 'bazaar', 'inside', '<MISSING>', 'of', 'a volume', '<

In [14]:
def accuracy(X, Y, model):
    
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            for p, t in zip(most_likely_tags, actual_tags):
                if(p.find(t)!=-1):
                    correct += 1
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

In [15]:
def accuracy_new(X, Y, model):
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        
        try:
            most_likely_tags = simplify_decoding(observations, model)
            for p in most_likely_tags:
                total_predictions+=1
                if "<MISSING>" not in p:
                    correct+=1
        except:
            pass
        
#         total_predictions += len(observations)
    return correct/total_predictions

In [16]:
def accuracy_sen(X, Y, model,rate):
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        
        try:
            most_likely_tags = simplify_decoding(observations, model)
            total_predictions+=1
            find=total=0
            for p in most_likely_tags:               
                total+=1
                if "<MISSING>"  in p:
                    find+=1
            if find/total<=rate:
                correct+=1
                
                    
        except:
            pass
        
#         total_predictions += len(observations)
    return correct/total_predictions

In [16]:
mfc_training_acc = accuracy(cn_words, en_train, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

training accuracy mfc_model: 21.89%


In [17]:
mfc_testing_acc = accuracy(cn_test, en_test, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

testing accuracy mfc_model: 15.00%


In [18]:
mfc_testing_acc = accuracy_new(cn_test, en_test, mfc_model)
print("testing accuracy mfc_model(not missing): {:.2f}%".format(100 * mfc_testing_acc))

testing accuracy mfc_model(not missing): 93.80%


In [19]:
mfc_testing_acc = accuracy_new(cn_words, en_train, mfc_model)
print("testing accuracy mfc_model(not missing): {:.2f}%".format(100 * mfc_testing_acc))

testing accuracy mfc_model(not missing): 94.95%


In [19]:
rate=0.2

In [21]:
mfc_testing_acc = accuracy_sen(cn_words, en_train, mfc_model,rate)
print("testing accuracy mfc_model(not missing rate<="+str(rate)+" per sentence): {:.2f}%".format(100 * mfc_testing_acc))

testing accuracy mfc_model(not missing rate<=0.2 per sentence): 96.60%


In [20]:
mfc_testing_acc = accuracy_sen(cn_test, en_test, mfc_model,rate)
print("testing accuracy mfc_model(not missing rate<="+str(rate)+" per sentence): {:.2f}%".format(100 * mfc_testing_acc))

testing accuracy mfc_model(not missing rate<=0.2 per sentence): 90.00%



* accuracy_new:单词不丢失即算命中
* accuracy:完全匹配原版翻译
* accuracy_sen 单句中missing概率不超过rate

* accuracy_new： not missing means hit
* accuracy: Follow the original translation exactly
* accuracy_sen: missing in sentence not beyond the rate

# hmm

In [23]:
# read in data 
import json
l=0

# training set
cn_train=[]
en_train=[]
with open('data/translation2019zh_train.json', encoding='utf-8') as f:
    while l<100000:
        line=f.readline()
        j=json.loads(line)
        cn_train.append(j["chinese"])
        en_train.append(j["english"])
        l+=1
    f.close()
    
# test set
l=0
cn_test=[]
en_test=[]
with open('data/translation2019zh_valid.json', encoding='utf-8') as f:
    while l<10:
        line=f.readline()
        j=json.loads(line)
        cn_test.append(j["chinese"])
        en_test.append(j["english"])
        l+=1
    f.close()

* n	普通名词	f	方位名词	s	处所名词	t	时间
* nr	人名	ns	地名	nt	机构名	nw	作品名
* nz	其他专名	v	普通动词	vd	动副词	vn	名动词
* a	形容词	ad	副形词	an	名形词	d	副词
* m	数量词	q	量词	r	代词	p	介词
* c	连词	u	助词	xc	其他虚词	w	标点符号
* PER	人名	LOC	地名	ORG	机构名	TIME	时间

In [24]:
import jieba
import jieba.posseg as pseg
def seperate(cn_list):
    cn_words=[]
    cn_words_flag=[]
    for s in cn_list:
        words=[]
        wflag=[]
        w=pseg.cut(s)
        for word,flag in w:
            if(word==' '):
                continue
            words.append(word)
            wflag.append(flag)
        cn_words.append(words)
        cn_words_flag.append(wflag)
    return cn_words,cn_words_flag
    
cn_words,cn_flag = seperate(cn_train)
cn_words[0],cn_flag[0]


(['为了',
  '更好',
  '的',
  '锐度',
  '，',
  '但是',
  '附带',
  '的',
  '会',
  '多一些',
  '颗粒',
  '度',
  '，',
  '可以',
  '使用',
  '这个',
  '显影剂',
  '的',
  '1',
  '：',
  '1',
  '稀释液',
  '。'],
 ['p',
  'd',
  'uj',
  'n',
  'x',
  'c',
  'n',
  'uj',
  'n',
  'm',
  'n',
  'zg',
  'x',
  'c',
  'v',
  'r',
  'n',
  'uj',
  'm',
  'x',
  'x',
  'n',
  'x'])

In [25]:
flagset=[]
for flags in cn_flag:
    cleanflag=[]
    for flag in flags:
        if(flag.startswith('t') or flag.startswith('n') or flag.startswith('s') or flag.startswith('f') or flag.startswith('q')or flag.startswith('m')or flag.startswith('r')or flag.startswith('b')or flag.startswith('z')):
            cleanflag.append("n")
            continue
        if(flag.startswith('v') ):
            cleanflag.append("v")
            continue
        if(flag.startswith('a') ):
            cleanflag.append("adj")
            continue
        if(flag.startswith('d') ):
            cleanflag.append("adv")
            continue
        if(flag.startswith('p') ):
            cleanflag.append("prep")
            continue
        if(flag.startswith('c') ):
            cleanflag.append("c")
            continue
        if(flag.startswith('u') or flag.startswith('e')or flag.startswith('y')or flag.startswith('o')or flag.startswith('h')or flag.startswith('k')):
            cleanflag.append("u")
            continue
        if(flag.startswith('w') or flag.startswith('x')):
            cleanflag.append("w")
            continue
    flagset.append(cleanflag)
len(flagset[0]),len(cn_words[0]),flagset[0],cn_flag[0],cn_words[0]

(23,
 23,
 ['prep',
  'adv',
  'u',
  'n',
  'w',
  'c',
  'n',
  'u',
  'n',
  'n',
  'n',
  'n',
  'w',
  'c',
  'v',
  'n',
  'n',
  'u',
  'n',
  'w',
  'w',
  'n',
  'w'],
 ['p',
  'd',
  'uj',
  'n',
  'x',
  'c',
  'n',
  'uj',
  'n',
  'm',
  'n',
  'zg',
  'x',
  'c',
  'v',
  'r',
  'n',
  'uj',
  'm',
  'x',
  'x',
  'n',
  'x'],
 ['为了',
  '更好',
  '的',
  '锐度',
  '，',
  '但是',
  '附带',
  '的',
  '会',
  '多一些',
  '颗粒',
  '度',
  '，',
  '可以',
  '使用',
  '这个',
  '显影剂',
  '的',
  '1',
  '：',
  '1',
  '稀释液',
  '。'])

In [26]:
with open("json.txt","w+",encoding="utf-8") as f:
    i=0
    for flag,sentence in zip(flagset,cn_words):
        f.write("sentence12-"+str(i)+"\n")
        for fl,w in zip(flag,sentence):
            f.write(w+"\t"+fl+"\n")
        f.write("\n")
        i+=1

In [27]:
def read_data(filename):
    """Read tagged sentence data"""
    with open(filename, 'r', encoding='utf-8') as f:
        sentence_lines = [l.split("\n") for l in f.read().split("\n\n")]
#         print(type( (Sentence(*zip(*[l.strip().split("\t") for l in s[1:]]))) for s in sentence_lines if s[0]))
        a={}
        for s in sentence_lines:
            if s[0]:
                try:
#                     print(list(s))
                    a[s[0]]=Sentence(*zip(*[l.split("\t") for l in s[1:]]))
                except TypeError:
                    ;
#                     print(list(s))
#             
        return OrderedDict(a)
        
            
def read_tags(filename):
    """Read a list of word tag classes"""
    with open(filename, 'r', encoding='utf-8') as f:
        tags = f.read().split("\n")
    return frozenset(tags)

Sentence = namedtuple("Sentence", "words tags")

In [28]:
sentences=read_data("json.txt")

In [29]:
class Dataset(namedtuple("_Dataset", "sentences keys vocab X tagset Y training_set testing_set N stream")):
    def __new__(cls, tagfile, datafile, train_test_split=0.8, seed=112890):
        tagset = read_tags(tagfile)
        sentences = read_data(datafile)
        keys = tuple(sentences.keys())
        wordset = frozenset(chain(*[s.words for s in sentences.values()]))
        word_sequences = tuple([sentences[k].words for k in keys])
        tag_sequences = tuple([sentences[k].tags for k in keys])
        N = sum(1 for _ in chain(*(s.words for s in sentences.values())))
        
        # split data into train/test sets
        _keys = list(keys)
        if seed is not None: random.seed(seed)
        random.shuffle(_keys)
        split = int(train_test_split * len(_keys))
        training_data = Subset(sentences, _keys[:split])
        testing_data = Subset(sentences, _keys[split:])
        stream = tuple(zip(chain(*word_sequences), chain(*tag_sequences)))
        return super().__new__(cls, dict(sentences), keys, wordset, word_sequences, tagset,
                               tag_sequences, training_data, testing_data, N, stream.__iter__)

    def __len__(self):
        return len(self.sentences)

    def __iter__(self):
        return iter(self.sentences.items())
    
    
class Subset(namedtuple("BaseSet", "sentences keys vocab X tagset Y N stream")):
    def __new__(cls, sentences, keys):
        word_sequences = tuple([sentences[k].words for k in keys])
        tag_sequences = tuple([sentences[k].tags for k in keys])
        wordset = frozenset(chain(*word_sequences))
        tagset = frozenset(chain(*tag_sequences))
        N = sum(1 for _ in chain(*(sentences[k].words for k in keys)))
        stream = tuple(zip(chain(*word_sequences), chain(*tag_sequences)))
        return super().__new__(cls, {k: sentences[k] for k in keys}, keys, wordset, word_sequences,
                               tagset, tag_sequences, N, stream.__iter__)

    def __len__(self):
        return len(self.sentences)

    def __iter__(self):
        return iter(self.sentences.items())

In [30]:
data = Dataset("tags.txt", "json.txt", train_test_split=1)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 100000 sentences in the corpus.
There are 100000 sentences in the training set.
There are 0 sentences in the testing set.


## data prepare, we got flagset as class and cn_words

In [31]:
def unigram_counts(sequences):
    return Counter(sequences)

tags = [tag for i, (word, tag) in enumerate(data.training_set.stream())]
tag_unigrams = unigram_counts(tags)
tag_unigrams

Counter({'n': 829115,
         'adv': 89254,
         'v': 407766,
         'w': 301557,
         'adj': 57043,
         'u': 209480,
         'prep': 88516,
         'c': 77174})

In [32]:
def bigram_counts(sequences):
    return Counter(sequences)

tags = [tag for i, (word, tag) in enumerate(data.stream())]
o = [(tags[i],tags[i+1]) for i in range(0,len(tags)-2,2)]
tag_bigrams = bigram_counts(o)
tag_bigrams 

Counter({('prep', 'adv'): 817,
         ('u', 'n'): 60393,
         ('w', 'c'): 15180,
         ('n', 'u'): 49889,
         ('n', 'n'): 137328,
         ('v', 'n'): 80198,
         ('n', 'w'): 83926,
         ('w', 'n'): 71597,
         ('adv', 'prep'): 3583,
         ('v', 'w'): 31827,
         ('adj', 'n'): 7593,
         ('w', 'w'): 11562,
         ('n', 'v'): 82155,
         ('u', 'adj'): 3571,
         ('u', 'u'): 5214,
         ('w', 'prep'): 11067,
         ('adv', 'n'): 7869,
         ('w', 'v'): 24451,
         ('n', 'prep'): 15637,
         ('w', 'adv'): 7850,
         ('v', 'v'): 35918,
         ('prep', 'n'): 31334,
         ('n', 'c'): 15233,
         ('n', 'adv'): 21513,
         ('u', 'w'): 12448,
         ('u', 'v'): 18711,
         ('v', 'u'): 30024,
         ('n', 'adj'): 9246,
         ('c', 'prep'): 2619,
         ('c', 'n'): 17876,
         ('w', 'adj'): 3038,
         ('w', 'u'): 6065,
         ('adv', 'v'): 20257,
         ('adj', 'u'): 8748,
         ('adj', 'ad

In [33]:
def starting_counts(sequences):
    return Counter(sequences)

tags = [tag for i, (word, tag) in enumerate(data.stream())]
starts_tag = [i[0] for i in data.Y]
tag_starts = starting_counts(starts_tag)
tag_starts

Counter({'prep': 10796,
         'n': 56616,
         'w': 2689,
         'v': 12671,
         'c': 9369,
         'u': 2358,
         'adj': 1821,
         'adv': 3680})

In [34]:
def ending_counts(sequences):    
    return Counter(sequences)

end_tag = [i[len(i)-1] for i in data.Y]
tag_ends = ending_counts(end_tag)
tag_ends

Counter({'w': 98747,
         'n': 864,
         'v': 210,
         'u': 110,
         'adj': 42,
         'prep': 4,
         'adv': 18,
         'c': 5})

In [35]:
end_tag = [i[len(i)-2] for i in data.Y]
tag_ends = ending_counts(end_tag)
tag_ends

Counter({'n': 55469,
         'w': 4960,
         'u': 8914,
         'v': 24424,
         'adj': 3945,
         'adv': 1473,
         'prep': 321,
         'c': 494})

In [36]:
def pair_counts(tags, words):
    d = defaultdict(lambda: defaultdict(int))
    for tag, word in zip(tags, words):
        d[tag][word] += 1
    return d


In [37]:
hmm_model = HiddenMarkovModel(name="base-hmm-tagger")

tags = [tag for i, (word, tag) in enumerate(data.stream())]
words = [word for i, (word, tag) in enumerate(data.stream())]

tags_count = unigram_counts(tags)
tag_words_count = pair_counts(tags, words)

starting_tag_list = [i[0] for i in data.Y]
#ending_tag_list = [i[-1] if len(i)==1 else i[-2] for i in data.Y]
#ending_tag_list = [i[-1] for i in data.Y]
ending_tag_list = [i[len(i)-2] for i in data.Y]

starting_tag_count = starting_counts(starting_tag_list) #the number of times a tag occured at the start
ending_tag_count = ending_counts(ending_tag_list)       #the number of times a tag occured at the end

tag_words_count

defaultdict(<function __main__.pair_counts.<locals>.<lambda>()>,
            {'prep': defaultdict(int,
                         {'为了': 839,
                          '把': 2372,
                          '在': 24926,
                          '亚历山大': 1,
                          '从未': 5,
                          '与': 5809,
                          '通过': 2321,
                          '给': 1591,
                          '用': 2151,
                          '以': 2845,
                          '于': 2108,
                          '当': 731,
                          '不': 33,
                          '，': 1288,
                          '从': 3275,
                          '被': 3548,
                          '对': 8111,
                          '由': 1379,
                          '关于': 575,
                          '夫妻': 1,
                          '模特': 1,
                          '针对': 473,
                          '为': 5520,
                          '因': 419,
                 

In [38]:
to_pass_states = []
for tag, words_dict in tag_words_count.items():
    total = float(sum(words_dict.values()))
    distribution = {word: count/total for word, count in words_dict.items()}
    tag_emissions = DiscreteDistribution(distribution)
    tag_state = State(tag_emissions, name=tag)
    to_pass_states.append(tag_state)

In [39]:
hmm_model.add_states() 

In [40]:
start_prob={}

for tag in tags:
    start_prob[tag] = starting_tag_count[tag] / tags_count[tag]

for tag_state in to_pass_states :
    hmm_model.add_transition(hmm_model.start, tag_state, start_prob[tag_state.name])  

In [41]:
end_prob={}

for tag in tags:
    end_prob[tag] = ending_tag_count[tag]/tags_count[tag]
    
for tag_state in to_pass_states :
    hmm_model.add_transition(tag_state, hmm_model.end, end_prob[tag_state.name])

In [42]:
transition_prob_pair={}

for key in tag_bigrams.keys():
    transition_prob_pair[key] = tag_bigrams.get(key)/tags_count[key[0]]

i=0
for tag_state in to_pass_states:
    for next_tag_state in to_pass_states:
        i+=1
     
        try:
            hmm_model.add_transition(tag_state, next_tag_state, transition_prob_pair[(tag_state.name, next_tag_state.name)])
        except:
            print(tag_state.name, next_tag_state.name)

In [43]:
hmm_model.bake()

In [44]:
hmmdict={}
# read in dict
import pandas as pd
import re

def hmmdict():
    dic={}
    dictionary = pd.read_csv('dict/all.txt',header = None)
    for row in dictionary.itertuples():
        dictag={}
        translations=row[2][1:-1]       
        wordTags = row[3][1:-1]
        translations =translations.split(",")       
        wordTags = wordTags.split(",")  
        tags1=[]

        for tags,trans in zip(wordTags,translations):
            tags=tags.replace(".","").replace("'","").replace(" ","")
            if tags=="vi":
                tags='v'
            if tags=='vt':
                tags='v'
            trans=trans.replace(".","").replace("'","").replace(" ","")
#             print(tags,trans)
            dictag[tags]=trans
        dic[row[1]]=dictag
#             if trans!="" and trans!=" ":
#                 result.append(trans)
#         dic[row[1]]=result
    return dic

hmmdict=hmmdict()


In [91]:
punctuation = "∶５ｓ℃:\[\]―〈〉☆<>②!@#$%^&*()_+=,.\-．─\"·•，！\'/?？。＂＃＄％＆＇（）＊＋－／\：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏"
re_punctuation = "[{}]+".format(punctuation)
def getword(word,tag):
    if word==None or word.startswith(" "):
        return ""

    if(tag=='c'):
        return ""
            # look up dict, get its english word(list)
    if(re.match(re_punctuation, word, flags=0)):
        return "<X>"
    if(re.match("[a-zA-Z0-9]",word,flags=0)):
        return word
    rew=""
    try:
        rew=hmmdict[word][list(hmmdict[word])[0]]
        
    except:
        try:
            rew=hmmdict[word]["misc"]
            
        except:
            try:
                rew=hmmdict[word][tag]
            except:
                try:
                    for w in word:
                        rew+=endict[w][0]
                except:
                    rew= "<Missing>"
    if(tag==None):
        rew= "<"+rew+">"
    return rew

In [92]:
def replace_unknown(sequence):
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):    
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]

In [94]:
def translatehmm(wordlist):
    trans=[]
    for tag,word in zip(simplify_decoding(wordlist, hmm_model),wordlist):
#         print(word)
        w=getword(word,tag)
        if w != "":
            trans.append(w)
#         print(word,tag)
        
    return trans

In [95]:
# for key in data.testing_set.keys[:3]:
#     print("Sentence Key: {}\n".format(key))
#     print("Sentence: {}\n".format(data.sentences[key].words))
#     print("Predicted labels:\n-----------------")
#     print(translatehmm(key))
#     print()
#     print("Actual labels:\n--------------")
#     print(es)
#     print("\n")
# print(cn_test)
for sentence,es in zip(cn_sep_test[0:5],en_test[0:5]):
    #     sentenceinput=re.sub("[a-zA-Z0-9]","",sentenceinput)
    
    print("Sentence:",sentence)
    print("Predicted labels:\n-----------------")
    print(translatehmm(sentence))
    print("Actual labels:\n--------------")
    print(es)
    print("\n")

Sentence: ['美国', '缓慢', '地', '开始', '倾听', '，', '但', '并非', '没有', '艰难曲折', '。']
Predicted labels:
-----------------
['America', 'tardiness', 'land', 'initiation', 'giveearto', '<X>', 'without', 'difficultdifficultsongdiscount', '<X>']
Actual labels:
--------------
Slowly and not without struggle, America began to listen.


Sentence: ['直到', '四年', '前', '我', '才', '有', '了', '一本', '词典', '。', '我', '使用', '的', '是', '用', '89', '美分', '在', 'K', '市场', '里', '买来', '的', '一本', '韦氏', '小', '词典', '。', '我', '从来不', '使用', '单词', '处理程序', '。']
Predicted labels:
-----------------
['till', 'quadrennium', 'before', 'I', 'ability', 'adsum', 'end', 'avolume', 'dictionaries', '<X>', 'I', 'use', 'of', 'are', 'use', '89', 'beautyminute', 'in', 'K', 'bazaar', 'inside', 'purchasecome', 'of', 'avolume', 'leathersurname', 'little', 'dictionaries', '<X>', 'I', 'never', 'use', 'word', 'processingprogram', '<X>']
Actual labels:
--------------
I didn't own a Thesaurus until four years ago and I use a small Webster's dictionary tha